In [1]:
import os
import json
import torch
import torch.nn as nn    
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision import models
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
class RetinaFundusDataset(Dataset):
    def __init__(self, images_dir, annotations_dir=None, transform=None):
        self.images_dir = images_dir
        self.annotations_dir = annotations_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(images_dir) if f.endswith('.jpg') or f.endswith('.png')]

    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        img_name = os.path.join(self.images_dir, self.image_files[idx])

        # Check if annotations are available
        if self.annotations_dir:
            annotation_name = os.path.join(self.annotations_dir, self.image_files[idx].replace('.jpg', '.json').replace('.png', '.json'))
            # Load annotations (bounding boxes)
            if os.path.exists(annotation_name):
                with open(annotation_name, 'r') as f:
                    annotations = json.load(f)

                # Extract bounding boxes for optic disc and optic cup
                disc_box = annotations['optic_disc']
                cup_box = annotations['optic_cup']

                # Extract height and width of the bounding boxes
                disc_height = disc_box['height']
                disc_width = disc_box['width']
                cup_height = cup_box['height']
                cup_width = cup_box['width']

                # Targets: height and width of optic disc and optic cup
                target = [disc_height, disc_width, cup_height, cup_width]
            else:
                target = torch.zeros(4)  # Default to zero if annotation file is missing
        else:
            # No annotations in test set
            target = torch.zeros(4)  # Dummy target for test set, no annotations required

        # Load image
        image = Image.open(img_name).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(target, dtype=torch.float32)

In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
train_images_dir = 'data/train'
train_annotations_dir = 'annotations'
test_images_dir = 'data/test'
# test_annotations_dir = 'annotations'

# Create dataset and dataloaders
train_dataset = RetinaFundusDataset(train_images_dir, annotations_dir=train_annotations_dir, transform=transform)
test_dataset = RetinaFundusDataset(test_images_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [5]:
class RetinaModel(nn.Module):
    def __init__(self):
        super(RetinaModel, self).__init__()
        self.resnet = models.resnet18(pretrained=True)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 4)  # 4 outputs for height/width of optic disc and cup
    
    def forward(self, x):
        return self.resnet(x)

In [6]:
model = RetinaModel()
criterion = nn.MSELoss()  # Mean Squared Error for regression task
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}')

C:\Users\mdans\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\mdans\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\mdans\AppData\Local\Temp\ipykernel_14244\1448542527.py:46: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceT

Epoch 1/10, Loss: 0.8572620277603468
Epoch 2/10, Loss: 0.018293403865148623
Epoch 3/10, Loss: 0.005204361941044529
Epoch 4/10, Loss: 0.0023224879793512323
Epoch 5/10, Loss: 0.0010766345561326791
Epoch 6/10, Loss: 0.0006280640081968158
Epoch 7/10, Loss: 0.00040051076697030413
Epoch 8/10, Loss: 0.00021040365390945226
Epoch 9/10, Loss: 0.00011449529847595841
Epoch 10/10, Loss: 9.137556238177542e-05


In [7]:
model.eval()
total = 0
correct = 0
disc_height_lst=[]
cup_height_lst=[]
disc_width_lst=[]
cup_width_lst=[]
area_disc=[]
area_cup=[]
with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        total += 1

        # Print bounding box predictions (height and width)
        for i, output in enumerate(outputs):
            # Extract predicted bounding boxes
            disc_height, disc_width, cup_height, cup_width = output

            # Calculate the area of optic disc and optic cup
            disc_area = disc_height * disc_width
            cup_area = cup_height * cup_width

            print(f"Image {i+1} - Predicted bounding boxes:")
            print(f"Optic Disc - Height: {disc_height:.2f}, Width: {disc_width:.2f}, Area: {disc_area:.2f}")
            disc_height_lst.append(abs(disc_height))
            disc_width_lst.append(abs(disc_width))
            area_disc.append(abs(disc_area))
            print(f"Optic Cup - Height: {cup_height:.2f}, Width: {cup_width:.2f}, Area: {cup_area:.2f}")
            cup_height_lst.append(abs(cup_height))
            cup_width_lst.append(abs(cup_width))
            area_cup.append(abs(cup_area))
    
    print(f"Test Loss: {loss.item()}")

# Save the model
torch.save(model.state_dict(), 'retina_model.pth')

C:\Users\mdans\AppData\Local\Temp\ipykernel_14244\1448542527.py:46: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return image, torch.tensor(target, dtype=torch.float32)


Image 1 - Predicted bounding boxes:
Optic Disc - Height: -0.09, Width: -0.21, Area: 0.02
Optic Cup - Height: 0.07, Width: -0.08, Area: -0.01
Image 2 - Predicted bounding boxes:
Optic Disc - Height: -0.12, Width: -0.24, Area: 0.03
Optic Cup - Height: 0.05, Width: -0.08, Area: -0.00
Image 3 - Predicted bounding boxes:
Optic Disc - Height: -0.09, Width: -0.21, Area: 0.02
Optic Cup - Height: 0.07, Width: -0.08, Area: -0.01
Image 4 - Predicted bounding boxes:
Optic Disc - Height: -0.09, Width: -0.20, Area: 0.02
Optic Cup - Height: 0.06, Width: -0.07, Area: -0.00
Image 5 - Predicted bounding boxes:
Optic Disc - Height: -0.13, Width: -0.25, Area: 0.03
Optic Cup - Height: 0.07, Width: -0.09, Area: -0.01
Image 6 - Predicted bounding boxes:
Optic Disc - Height: -0.11, Width: -0.23, Area: 0.03
Optic Cup - Height: 0.04, Width: -0.08, Area: -0.00
Image 7 - Predicted bounding boxes:
Optic Disc - Height: -0.12, Width: -0.25, Area: 0.03
Optic Cup - Height: 0.05, Width: -0.08, Area: -0.00
Image 8 - Pre

In [8]:
print(type(disc_height_lst[0]))
print(disc_width_lst)
print(area_disc)
print(cup_height_lst)
print(cup_width_lst)
print(area_cup)

<class 'torch.Tensor'>
[tensor(0.2119), tensor(0.2409), tensor(0.2119), tensor(0.1972), tensor(0.2481), tensor(0.2291), tensor(0.2480), tensor(0.2291), tensor(0.2061), tensor(0.1718), tensor(0.2151), tensor(0.1975), tensor(0.2483), tensor(0.1521), tensor(0.2171), tensor(0.1521), tensor(0.2405), tensor(0.2126), tensor(0.1965), tensor(0.2034), tensor(0.1942), tensor(0.2032), tensor(0.2370), tensor(0.2422), tensor(0.2262), tensor(0.2422), tensor(0.2262), tensor(0.2775), tensor(0.2232), tensor(0.1672), tensor(0.2104), tensor(0.2793), tensor(0.2233), tensor(0.2793), tensor(0.2200), tensor(0.2531), tensor(0.2371), tensor(0.2259), tensor(0.2291), tensor(0.2089), tensor(0.2742), tensor(0.2426), tensor(0.2376), tensor(0.2295), tensor(0.2667), tensor(0.2217), tensor(0.2179), tensor(0.2178), tensor(0.1743), tensor(0.2393), tensor(0.2233), tensor(0.2294), tensor(0.2273), tensor(0.2294), tensor(0.2406), tensor(0.2464), tensor(0.2241), tensor(0.2464), tensor(0.2290), tensor(0.2319), tensor(0.2241), 

In [9]:
def convert(lst):
    for i in range(len(lst)):
        lst[i]=f"{lst[i]:.5f}"
        lst[i] = float(lst[i])
    return lst

In [10]:
disc_height_lst=convert(disc_height_lst)
disc_width_lst=convert(disc_width_lst)
area_disc=convert(area_disc)
cup_height_lst=convert(cup_height_lst)
cup_width_lst=convert(cup_width_lst)
area_cup=convert(area_cup)
print(type(disc_height_lst))
print(type(disc_height_lst[0]))
print(disc_height_lst)

<class 'list'>
<class 'float'>
[0.09326, 0.12175, 0.09326, 0.086, 0.12917, 0.11329, 0.11526, 0.11329, 0.09357, 0.09076, 0.11716, 0.08978, 0.1225, 0.07122, 0.10689, 0.07122, 0.11635, 0.10517, 0.09956, 0.08796, 0.10139, 0.10148, 0.12771, 0.1151, 0.11061, 0.1151, 0.11061, 0.13687, 0.09932, 0.09282, 0.11736, 0.13426, 0.11656, 0.13426, 0.11355, 0.11155, 0.10493, 0.10646, 0.11134, 0.09848, 0.13939, 0.11898, 0.11739, 0.11976, 0.13889, 0.10955, 0.11096, 0.10568, 0.09367, 0.11951, 0.12093, 0.11484, 0.10021, 0.11484, 0.10416, 0.12099, 0.10824, 0.12099, 0.1097, 0.10745, 0.10824, 0.10745, 0.1097, 0.10824, 0.1097, 0.12896, 0.12671, 0.1197, 0.1189, 0.11829, 0.12113, 0.11829, 0.11824, 0.12113, 0.11824, 0.12192, 0.11963, 0.12192, 0.11848, 0.11963, 0.11848, 0.10088, 0.10022, 0.10913, 0.10106, 0.10913, 0.09778, 0.10106, 0.09778, 0.10962, 0.10433, 0.11006, 0.10986, 0.11006, 0.11172, 0.10986, 0.11172, 0.10655, 0.10724, 0.09959, 0.10655, 0.09959, 0.10724]


In [11]:
def diameter(width,height):
    l=[]
    for i in range(len(width)):
       l.append((height[i]+width[i])/2)
    return l

In [12]:
dic = {"disc_diameter":diameter(disc_width_lst,disc_height_lst), "cup_diameter":diameter(cup_width_lst,cup_height_lst)}
data = pd.DataFrame(dic)
data["disc_area"] = data["disc_diameter"].apply(lambda x: (x/2)**2*np.pi)
data["cup_area"] = data["cup_diameter"].apply(lambda x: (x/2)**2*np.pi)
data.to_csv("retina_insights.csv")
data.head()

,disc_diameter,cup_diameter,disc_area,cup_area
0,0.152590,0.074295,0.018287,0.004335
1,0.181325,0.065600,0.025823,0.003380
2,0.152590,0.074295,0.018287,0.004335
3,0.141580,0.065135,0.015743,0.003332
4,0.188640,0.081360,0.027948,0.005199
